In [1]:
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import nibabel as nib
import matplotlib.pyplot as plt
from dataset import CarotidDataset
from unet import UNet

from torchvision import transforms

In [2]:
data = CarotidDataset()

epochs = 2
BATCH_SIZE = 4
net = UNet(in_channels=3, n_classes=1, padding=True, up_mode='upsample')
device = torch.device("mps")
#device = torch.device("cpu")
trainloader = torch.utils.data.DataLoader(data, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
step_count = 0
loss = nn.CrossEntropyLoss()

transforms = torch.nn.Sequential(
    transforms.CenterCrop((736, 704))
)

In [ ]:
for epoch in range(epochs):
    print(f'Epoch {epoch}/{epochs - 1}')
    print('-' * 10)
    net.train()
    net = net.to(device)
    train_running_loss = 0.0
    train_running_corrects = 0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        #inputs = inputs.float()
        #labels = labels.float()
        #inputs = inputs.reshape([BATCH_SIZE,1,749,709])
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        preds = net(inputs)
        #train_loss = F.cross_entropy(preds, y)
        #train_loss = loss(preds, transforms(labels.squeeze(1)))
        train_loss = dice_loss(preds,labels)
        print(train_loss)
        train_loss.backward()
        optimizer.step()
        step_count += 1
        train_running_loss += train_loss.item()

In [3]:
net.train()
net = net.to(device)
train_running_loss = 0.0
train_running_corrects = 0

In [4]:
inputs, labels = next(iter(trainloader))
inputs = inputs.to(device)
labels = labels.to(device)
optimizer.zero_grad()
preds = net(inputs)
preds = F.pad(preds, (3,2,7,6))

In [5]:
preds.shape

torch.Size([4, 1, 749, 709])

In [6]:
preds.view(-1).shape

torch.Size([2124164])

In [7]:
labels.shape

torch.Size([4, 1, 749, 709])

In [ ]:
train_loss = dice_loss(preds,labels)

In [8]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice
dice_loss = DiceLoss()

In [9]:
train_loss = dice_loss(preds, labels)

In [10]:
train_loss

tensor(0.0255, device='mps:0', grad_fn=<RsubBackward1>)